# New keywords

### Create dataset from current audio folder

In [1]:
import os
import tqdm
from pathlib import Path
import pandas as pd
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

PATH_TO_AUDIO = "google_speech_recognition_v2"

/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


source: https://colab.research.google.com/github/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb#scrollTo=-gh7fQ1XEpC7

In [2]:
data = []

for subdir, dirs, files in os.walk(PATH_TO_AUDIO):
    for file in files:
        if file.endswith(".wav"):
            name = file.split(".")[0]
            label = subdir.split("/")[-1] # label = subdir.split("\\")[-1] on windows
            path = os.path.join(subdir, file)
            data.append({
                "name": name,
                "path": path,
                "keyword": label,
            })

In [3]:
df = pd.DataFrame(data)
print("Labels: ", df["keyword"].unique())
print()
#df.groupby("keyword").count()[["path"]]

Labels:  ['go' 'six' 'up' 'happy' 'sheila' 'follow' 'wow' 'four' 'learn' 'forward'
 'house' 'zero' 'on' 'left' 'no' 'backward' 'right' 'bird' 'eight'
 'visual' 'marvin' 'bed' 'stop' 'nine' 'seven' 'five' 'yes' 'one'
 '_background_noise_' 'off' 'dog' 'two' 'three' 'tree' 'down' 'cat']



In [4]:
# we only choose the keywords we want
desired_keywords = ["follow", "go", "happy", "marvin", "stop", "down"]
df = df[df["keyword"].isin(desired_keywords)]
print("Labels: ", df["keyword"].unique())
print()
df.groupby("keyword").count()[["path"]]

Labels:  ['go' 'happy' 'follow' 'marvin' 'stop' 'down']



,path
keyword,
down,3917
follow,1579
go,3880
happy,2054
marvin,2100
stop,3872


In [5]:
idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["keyword"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
# speech = librosa.resample(np.asarray(speech), sr, 16_000) # audio is already at 16kHz
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 11171
      Label: stop



In [6]:
# create csv files to be used to load data
save_path = "gsr_v2_cleaned"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["keyword"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(13921, 3)
(3481, 3)


In [7]:
# Loading the created dataset using datasets

data_files = {
    "train": save_path+"/train.csv", 
    "validation": save_path+"/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 2432.19it/s]


Dataset csv downloaded and prepared to /home/guillaume/.cache/huggingface/datasets/csv/default-1847f1d03778b3e7/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 1244.05it/s]

Dataset({
    features: ['name', 'path', 'keyword'],
    num_rows: 13921
})
Dataset({
    features: ['name', 'path', 'keyword'],
    num_rows: 3481
})


In [8]:
labels = df["keyword"].unique()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["0"]

'go'

In [9]:
from transformers import AutoFeatureExtractor


model_checkpoint = "facebook/wav2vec2-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
input_column = "path"
output_column = "keyword"
max_duration = 1.0  # seconds
target_sampling_rate = feature_extractor.sampling_rate

def speech_file_to_array(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    label_list = list(label_list)
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

def preprocess_function(examples):
    audio_arrays = [speech_file_to_array(path) for path in examples[input_column]]
    target_list = [label_to_id(label, labels) for label in examples[output_column]]

    result = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True,
        padding=True 
    )

    # print(result)
    result["label"] = list(target_list)

    return result

In [11]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["path", "keyword", "name"], batched=True)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'label'],
        num_rows: 13921
    })
    validation: Dataset({
        features: ['input_values', 'label'],
        num_rows: 3481
    })
})

In [12]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_hid.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'projector.weight', 'projector.

In [13]:
batch_size = 16

In [14]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
metric = load_metric("accuracy")

/tmp/ipykernel_7735/2588679507.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [16]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

/home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 10/1085 [00:22<37:17,  2.08s/it] 

{'loss': 1.7921, 'learning_rate': 2.7522935779816517e-06, 'epoch': 0.05}


  2%|▏         | 20/1085 [00:42<36:31,  2.06s/it]

{'loss': 1.7822, 'learning_rate': 5.504587155963303e-06, 'epoch': 0.09}


  3%|▎         | 30/1085 [01:03<36:02,  2.05s/it]

...

                                                   
100%|██████████| 1085/1085 [40:01<00:00,  2.06s/it]

{'eval_loss': 0.06069877743721008, 'eval_accuracy': 0.9864981327204826, 'eval_runtime': 34.9516, 'eval_samples_per_second': 99.595, 'eval_steps_per_second': 6.237, 'epoch': 4.98}


100%|██████████| 1085/1085 [40:02<00:00,  2.21s/it]

{'train_runtime': 2402.5691, 'train_samples_per_second': 28.971, 'train_steps_per_second': 0.452, 'train_loss': 0.3366053492242839, 'epoch': 4.98}


TrainOutput(global_step=1085, training_loss=0.3366053492242839, metrics={'train_runtime': 2402.5691, 'train_samples_per_second': 28.971, 'train_steps_per_second': 0.452, 'train_loss': 0.3366053492242839, 'epoch': 4.98})

In [32]:
trainer.evaluate()

100%|██████████| 218/218 [00:34<00:00,  6.29it/s]


{'eval_loss': 0.06069877743721008,
 'eval_accuracy': 0.9864981327204826,
 'eval_runtime': 34.8562,
 'eval_samples_per_second': 99.867,
 'eval_steps_per_second': 6.254,
 'epoch': 4.98}

In [ ]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("anton-l/my-awesome-model")
model = AutoModelForAudioClassification.from_pretrained("anton-l/my-awesome-model")

### Prediction

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor_checkpoint = "facebook/wav2vec2-base"
audio_classification_checkpoint = "wav2vec2-base-finetuned-ks/checkpoint-1085"
feature_extractor = AutoFeatureExtractor.from_pretrained(feature_extractor_checkpoint)
config = AutoConfig.from_pretrained(feature_extractor_checkpoint)
sampling_rate = feature_extractor.sampling_rate
model = AutoModelForAudioClassification.from_pretrained(audio_classification_checkpoint).to(device)

In [40]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    # attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = scores
    # outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


STYLES = """
<style>
div.display_data {
    margin: 0 auto;
    max-width: 500px;
}
table.xxx {
    margin: 50px !important;
    float: right !important;
    clear: both !important;
}
table.xxx td {
    min-width: 300px !important;
    text-align: center !important;
}
</style>
""".strip()

def prediction(df_row):
    path, emotion = df_row["path"], df_row["emotion"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(STYLES + df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [41]:
# with pyaudio
import pyaudio
import wave
import tempfile
import os

CHUNK = 320  # number of audio samples per frame
FORMAT = pyaudio.paInt16  # audio format
CHANNELS = 1  # mono audio
RATE = 48000  # sampling rate in Hz
RECORD_SECONDS = 1  # duration of each recording in seconds
FILE_NAME = f"temp.wav"

def record_audio():
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK,
                    input_device_index=4)

    try:
        while True:
            frames = []  # to store audio frames

            for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                data = stream.read(CHUNK)
                frames.append(data)

            # write frames to temporary WAV file
            
            wav_filename =  FILE_NAME
            wf = wave.open(wav_filename, 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(frames))
            wf.close()

            # read contents of WAV file a

            yield wav_filename

    except KeyboardInterrupt:
        pass

    stream.stop_stream()
    stream.close()
    p.terminate()

In [44]:
labels

array(['go', 'happy', 'follow', 'marvin', 'stop', 'down'], dtype=object)

In [48]:
for wav_data in record_audio():
    # pass the WAV data to your keyword spotter here
    label = predict(wav_data, 16000)
    max = np.argmax(label)
    if max > 0.99:
        print(id2label[str(max)])
    else:
        print("not confident enough")
    

not confident enough
down
not confident enough
happy
happy
happy
not confident enough
not confident enough
happy
not confident enough
follow
happy
happy
happy
happy
not confident enough
happy
happy
happy
not confident enough
happy
happy
down
stop
not confident enough
not confident enough
stop
happy
not confident enough
down
not confident enough
not confident enough
happy
happy
happy
happy
happy
happy
happy
happy
not confident enough
happy
not confident enough
happy
happy
happy
happy
happy
happy
happy
not confident enough
happy
happy
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
not confident enough
happy
not confident enough
happy
not confident enough
happy
not confident enough
not confident enough
happy
not confident enough
not confident enough
not confident enough
not confident enough
happy
down
marvin
down
marvin
not confident enough
marvin
n